In [28]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from __future__ import print_function
from rdkit import Chem
from rdkit import Avalon
from rdkit.Avalon import pyAvalonTools

In [29]:
"""
Read in train and test as Pandas DataFrames
"""
df_train = pd.read_csv("train.gz", usecols=[0])

In [ ]:
df_train.head()

In [30]:
def split_mask(dftouse):
    itrain, itest = train_test_split(xrange(dftouse.shape[0]), train_size=0.001)
    mask=np.ones(dftouse.shape[0], dtype='int')
    mask[itrain]=1
    mask[itest]=0
    mask = (mask==1)
    return mask

In [32]:
def addFeatures(df):
    noFeatures = 512
    noMolecules = len(df_train['smiles'])
    prog1 = float(noMolecules)
    prog2 = float(noFeatures)
    print(prog1,prog2)
    for i in range(noFeatures):
        df[i] = pd.Series(np.zeros(noMolecules), index=df_train.index)
        print((i / prog2), end="\r")
    for i in range(noMolecules):
        print((i / prog1), end="\r")
        m = Chem.MolFromSmiles(df.smiles.iloc[i])
        fp = pyAvalonTools.GetAvalonFP(m)
        fpBit = fp.ToBitString()
        for j in range(noFeatures):
            featureBool = fpBit[j]
            df.iat[i,j+1] = featureBool
    return df

In [35]:
%%time
n = addFeatures(df_train)
n.to_csv('data.csv')

1000.0 512.0
Wall time: 54 s
